In [1]:
import numpy as np
from DMD import DMD
import matplotlib as mpl
import matplotlib.pyplot as plt
import os,time,sys
import h5py
import scipy as sci
import scipy.spatial.distance as ssd
import parseCIFTI as pC
from glob import glob
import h5py
import copy
from tqdm import tqdm_notebook as tqdm
from sklearn.decomposition import FastICA

#import utility functions. Notably includes:
#'binCortexData(x,nb=40)': take array of grayordinate data and convert to (flattened) binned space (split into nb bins)
#'flat2mat(x)': take flattened, binned vector and convert to matrix for visualization
#'mat2flat(x)': take matrix and convert to flat
#'initR()': "R" is a necessary global variable which gets defined in the above functions, this is a workaround to make sure it's defined; this should be changed
from hcp_dmd_utils import *
initR()

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#FRAME0S is a list of where each window should start (setting last argument to '4' means windows slide in increments of 4)
FRAME0S=np.arange(0,1200,4)
#number of modes to calculate in each window
nmodes=8
#length of window, in frames
nframes=32
FRAME0S=FRAME0S[(FRAME0S+nframes)<=1200]

#load list of 120 subjects to analyze
u120=np.genfromtxt('u120.txt').astype(int).astype(str)
ulist=u120

#define which runs to iterate over:
scandirs=['LR','RL']
runlabels=['REST1','REST2']
dL,dR,idx,scb,scl=pC.parseCoords()
npoint=dR.shape[0]+dL.shape[0]

#datapath: folder in which data is located
#datapath='./data/'
datapath='/hdd/HCP_DMD/HCP_DMD/data/'

## DMD Results

In [3]:
for runlabel in runlabels:
    for scandir in scandirs:
        fdir='./DMD_results/nmodes{:}_nframes{:}_{:}_{:}/'.format(nmodes,nframes,runlabel,scandir)
        for k,u in enumerate(ulist):
            if os.path.exists(fdir+'Modes_{:}.h5'.format(u)) is False:
                FT=[]
                PT=[]
                ux=[]
                jx=[]
                jno=[]
                Phi=[]
                fname=datapath+u+'/rfMRI_'+runlabel+'_'+scandir+'_Atlas.dtseries.nii'
                #only perform if the corresponding data file exists..
                if os.path.exists(fname) is True:
                    print '[{:} modes, {:} frames] {:}/{:} :'.format(nmodes,nframes,k+1,len(ulist))+fname
                    #load data
                    Xr=pC.loadGrayord(fname)
                    #restrict to cortical grayords
                    Xr=Xr[:npoint]
                    #iterate through windows
                    for j,frame0 in enumerate(tqdm(FRAME0S)):
                        #select window of data
                        X=Xr[:,frame0:(frame0+nframes)]
                        
                        #perform DMD on window of data, calculated 'nmodes' modes
                        phik,_,_,pt,ft,_=DMD(X,n=nmodes,ploton=False)
                        #only keep positive frequencies (drop conjugates)
                        phik=phik[:,ft>=0]
                        pt=pt[ft>=0]
                        ft=ft[ft>=0]

                        FT.append(ft)
                        PT.append(pt)
                        Phi.append(phik)
                        #record:
                        #subject number (ux),
                        #window number (jx),
                        #mode number within window (jno)
                        ux.append(k*np.ones((len(ft),)))
                        jx.append(j*np.ones((len(ft),)))
                        jno.append(np.arange(len(ft)))
                        

                    #Separate collected modes into magnitude, phase
                    Phi=np.absolute(np.hstack(Phi))
                    #stack frequencies, powers, etc.
                    freq=np.hstack(FT)
                    power=np.hstack(PT)
                    ux=np.hstack(ux)
                    jx=np.hstack(jx)
                    jno=np.hstack(jno)

                    #% SAVE RESULTS INTO h5 FILE
                    try:
                        os.mkdir('./DMD_results')
                    except:
                        pass
                    try:
                        os.mkdir(fdir)
                    except:
                        pass
                    #save frequency, power, subject/window number, etc.
                    with h5py.File(fdir+'{:}.h5'.format(u),'w') as hf:
                        hf.create_dataset("freq",data=freq)
                        hf.create_dataset("power",data=power)
                        hf.create_dataset("jx",data=jx)
                        hf.create_dataset("jno",data=jno)
                        hf.create_dataset("ux",data=ux)
                        hf.create_dataset("nmodes",data=nmodes)
                        hf.create_dataset("nframes",data=nframes)
                        hf.create_dataset("scandir",data=scandir)
                    #% APPLY SPATIAL BINNING TO ALL MODES
                    F=binCortexData(Phi)
                    F=F.T
                    #save binned magnitude/phase data in separate file
                    with h5py.File(fdir+'Modes_{:}.h5'.format(u),'w') as hf:
                        hf.create_dataset("F",data=F)
                else:
                    print('File does not exist: {:}'.format(fname))
                    
            else:
                print('{:} Already processed! Moving on...'.format(u))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


102513 Already processed! Moving on...
104012 Already processed! Moving on...
104416 Already processed! Moving on...
107321 Already processed! Moving on...
107422 Already processed! Moving on...
109123 Already processed! Moving on...
112516 Already processed! Moving on...
114217 Already processed! Moving on...
118124 Already processed! Moving on...
118225 Already processed! Moving on...
122822 Already processed! Moving on...
123925 Already processed! Moving on...
130619 Already processed! Moving on...
131217 Already processed! Moving on...
131722 Already processed! Moving on...
136732 Already processed! Moving on...
137229 Already processed! Moving on...
137633 Already processed! Moving on...
145127 Already processed! Moving on...
148335 Already processed! Moving on...
149842 Already processed! Moving on...
151425 Already processed! Moving on...
152831 Already processed! Moving on...
154936 Already processed! Moving on...
155938 Already processed! Moving on...
157437 Already processed!

HBox(children=(IntProgress(value=0, max=293), HTML(value=u'')))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


[8 modes, 32 frames] 34/120 :/hdd/HCP_DMD/HCP_DMD/data/168240/rfMRI_REST2_RL_Atlas.dtseries.nii


...

[8 modes, 32 frames] 119/120 :/hdd/HCP_DMD/HCP_DMD/data/987983/rfMRI_REST2_RL_Atlas.dtseries.nii


HBox(children=(IntProgress(value=0, max=293), HTML(value=u'')))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


[8 modes, 32 frames] 120/120 :/hdd/HCP_DMD/HCP_DMD/data/993675/rfMRI_REST2_RL_Atlas.dtseries.nii


HBox(children=(IntProgress(value=0, max=293), HTML(value=u'')))